# Targets for Abi

She's on at Magellan June 3 and 22, 2022, and needs second-half-night targets.

| field | ra | dec|
|------------|--------|----------|
|COSMOS-1    | 150.0  |  3.1     |
|COSMOS-2    | 149.22 | 1.75     |
|COSMOS-3    | 150.78 | 1.75     |
|DECaPS-East | 270.89 | -29.53   |
|ELAIS-E1    | 7.8746 | -43.0096 |
|ELAIS-E2    | 9.5    | -43.9980 |
|DECaPS-West | 116.32 | -26.25   |

## June 3

There are no new ELAIS targets yet.

COSMOS has set, as has DECaPS-West.

So we're looking for DECaPS-East targets.

In [ ]:
import psycopg2
import psycopg2.extras
import getpass
# from IPython.display import display
# import ipywidgets

import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
# import pandas as pd

from copy import deepcopy
import time
from astropy.time import Time

font = {'size' : 15}
mpl.rc('font', **font)

In [ ]:
dbuser = input("DB User: ")
dbpasswd = getpass.getpass("DB Password: ")

In [ ]:
db = psycopg2.connect(f"dbname='decat' user='{dbuser}' password='{dbpasswd}' host='decatdb.lbl.gov'")

db.autocommit = True # Makes the computer nicer about SQL syntax errors

cursor = db.cursor( cursor_factory = psycopg2.extras.DictCursor )

In [ ]:
### If you want to print table schema
# tables = ['exposures','subtractions','objects','objectrbs','candidates']
tables = ['exposures','subtractions','objects','candidates']
for table in tables:
    query = "SELECT column_name, data_type FROM information_schema.columns WHERE table_name=%s"
    cursor.execute( query, ( table, ))
    print( f"\nTABLE: {table}\n===========================" )
    for row in cursor:
        print( f"{row['column_name']:24s}  :  {row['data_type']:s}" )

### Look for recent candidates in DECaPS-East

In [ ]:
# t1 = time.time()

# query = ("SELECT id, ra, dec, created_at, modified FROM candidates "
#          "WHERE ra > 268 AND ra < 273 AND dec < -27 AND dec > -32")

# cursor.execute( query )
# results = np.array( cursor.fetchall() ).transpose()

# t2 = time.time()
# print(t2-t1)
# del t1,t2

# cand_id       = np.asarray( results[0] )
# cand_ra       = np.asarray( results[1], dtype='float' )
# cand_dec      = np.asarray( results[2], dtype='float' )
# cand_created  = np.asarray( results[3], dtype='str' )
# cand_modified = np.asarray( results[4], dtype='str' )
# del query, results

# print(len(cand_id))

In [ ]:
# print('Dates of DECaPS-East candidates, first and last created.')
# sx = np.argsort( cand_created )
# print(cand_created[sx[0]])
# print(cand_created[sx[-1]])
# print(' ')
# print('Dates of DECaPS-East candidates, first and last modified.')
# sx = np.argsort( cand_modified )
# print(cand_modified[sx[0]])
# print(cand_modified[sx[-1]])
# print(' ')
# print('There does not seem to be any DECaPS-East candidates from 2022A, started April 2022')

### Look for recent objects in DECaPS-East

In [ ]:
t1 = time.time()

query = ("SELECT o.id, o.ra, o.dec, o.candidate_id, o.subtraction_id, o.mag, o.magerr, "
         "rbs.rb, sub.exposure_id, exp.mjd, exp.filter "
         "FROM objects AS o "
         "JOIN objectrbs AS rbs ON o.id=rbs.object_id AND rbs.rbtype_id=1 "
         "JOIN subtractions AS sub ON sub.id=o.subtraction_id "
         "JOIN exposures AS exp ON exp.id=sub.exposure_id "
         "WHERE o.ra > 268 AND o.ra < 273 AND o.dec < -27 AND o.dec > -32 "
         "AND exp.mjd > 59680")
cursor.execute( query )
results = np.array( cursor.fetchall() ).transpose()

t2 = time.time()
print(t2-t1)
del t1,t2

obj_id     = np.asarray( results[0], dtype='int' )
obj_ra     = np.asarray( results[1], dtype='float' )
obj_dec    = np.asarray( results[2], dtype='float' )
obj_candid = np.asarray( results[3] )
obj_subid  = np.asarray( results[4], dtype='int' )
obj_mag    = np.asarray( results[5], dtype='float' )
obj_magerr = np.asarray( results[6], dtype='float' )
obj_rb     = np.asarray( results[7], dtype='float' )
obj_expid  = np.asarray( results[8], dtype='int' )
obj_mjd    = np.asarray( results[9], dtype='float' )
obj_filter = np.asarray( results[10] )

del query, results

print(len(obj_id))

## Find and remove objects for candidates with < 45 detections

In [ ]:
sx = np.argsort(obj_mjd)

temp1 = Time(obj_mjd[sx[0]], format='mjd')
temp2 = Time(obj_mjd[sx[-1]], format='mjd')

print('First object detected in DECaPS-East on: ',obj_mjd[sx[0]],temp1.iso)
print('Most recent object detected in DECaPS-East on: ',obj_mjd[sx[-1]],temp2.iso)

plt.hist( obj_mjd[sx], bins=1000, cumulative=True, histtype='step' )
# plt.xlim([59680,59740])
# plt.ylim([3e6,5e6])
plt.show()

del sx

In [ ]:
values, indices, counts = np.unique( obj_candid, return_inverse = True, return_counts = True )
print( 'obj_candid lengths = ', len(values), len(indices), len(counts) )
print( values )
print( indices )
print( counts )

plt.hist( counts, bins=20, log=True )
plt.show()

In [ ]:
tx = np.where(counts[indices] < 45)[0]
print(len(tx))

obj2_id     = np.delete( obj_id, tx )
obj2_ra     = np.delete( obj_ra, tx )
obj2_dec    = np.delete( obj_dec, tx )
obj2_candid = np.delete( obj_candid, tx )
obj2_subid  = np.delete( obj_subid, tx )
obj2_mag    = np.delete( obj_mag, tx )
obj2_magerr = np.delete( obj_magerr, tx )
obj2_rb     = np.delete( obj_rb, tx )
obj2_expid  = np.delete( obj_expid, tx )
obj2_mjd    = np.delete( obj_mjd, tx )
obj2_filter = np.delete( obj_filter, tx )

del tx

del obj_id, obj_ra, obj_dec, obj_candid, obj_subid
del obj_mag, obj_magerr, obj_rb, obj_expid, obj_mjd, obj_filter

In [ ]:
del values, indices, counts

## Make new candidates array

In [ ]:
cand_id, indices, cand_nobj = np.unique( obj2_candid, return_inverse = True, return_counts = True )
print( 'lengths = ', len(cand_id), len(indices), len(cand_nobj) )
print( cand_id )
print( indices )
print( cand_nobj )

In [ ]:
cand_gnobj = np.zeros( len(cand_id), dtype='int' )
cand_rnobj = np.zeros( len(cand_id), dtype='int' )
cand_inobj = np.zeros( len(cand_id), dtype='int' )

cand_gnep = np.zeros( len(cand_id), dtype='int' )
cand_rnep = np.zeros( len(cand_id), dtype='int' )
cand_inep = np.zeros( len(cand_id), dtype='int' )

cand_gnrep = np.zeros( len(cand_id), dtype='int' )
cand_rnrep = np.zeros( len(cand_id), dtype='int' )
cand_inrep = np.zeros( len(cand_id), dtype='int' )

cand_gmrb = np.zeros( len(cand_id), dtype='float' )
cand_rmrb = np.zeros( len(cand_id), dtype='float' )
cand_imrb = np.zeros( len(cand_id), dtype='float' )

cand_gpk = np.zeros( len(cand_id), dtype='float' )
cand_rpk = np.zeros( len(cand_id), dtype='float' )
cand_ipk = np.zeros( len(cand_id), dtype='float' )

cand_gamp = np.zeros( len(cand_id), dtype='float' )
cand_ramp = np.zeros( len(cand_id), dtype='float' )
cand_iamp = np.zeros( len(cand_id), dtype='float' )

for c, cid in enumerate(cand_id):
    ox = np.where( obj2_candid == cid )[0]
    
    gx = np.where( obj2_filter[ox] == 'g' )[0]
    rx = np.where( obj2_filter[ox] == 'r' )[0]
    ix = np.where( obj2_filter[ox] == 'i' )[0]
    
    cand_gnobj[c] = len(gx)
    cand_rnobj[c] = len(rx)
    cand_inobj[c] = len(ix)
    
    if len(gx) > 1:
        temp = np.unique( np.sort( np.floor( obj2_mjd[ox[gx]] ) ) )
        cand_gnep[c] = len(temp)
        tx = np.where( temp > 59715.)[0]
        cand_gnrep[c] = len(tx)
        del temp, tx
        cand_gmrb[c] = np.mean( obj2_rb[ox[gx]] )
        cand_gpk[c] = np.min( obj2_mag[ox[gx]] )
        cand_gamp[c] = np.max( obj2_mag[ox[gx]] ) - np.min( obj2_mag[ox[gx]] )
        
    if len(rx) > 1:
        temp = np.unique( np.sort( np.floor( obj2_mjd[ox[rx]] ) ) )
        cand_rnep[c] = len(temp)
        tx = np.where( temp > 59715.)[0]
        cand_rnrep[c] = len(tx)
        del temp, tx
        cand_rmrb[c] = np.mean( obj2_rb[ox[rx]] )
        cand_rpk[c] = np.min( obj2_mag[ox[rx]] )
        cand_ramp[c] = np.max( obj2_mag[ox[rx]] ) - np.min( obj2_mag[ox[rx]] )

    if len(ix) > 1:
        temp = np.unique( np.sort( np.floor( obj2_mjd[ox[ix]] ) ) )
        cand_inep[c] = len(temp)
        tx = np.where( temp > 59715.)[0]
        cand_inrep[c] = len(tx)
        del temp, tx
        cand_imrb[c] = np.mean( obj2_rb[ox[ix]] )
        cand_ipk[c] = np.min( obj2_mag[ox[ix]] )
        cand_iamp[c] = np.max( obj2_mag[ox[ix]] ) - np.min( obj2_mag[ox[ix]] )



In [ ]:
fig, ax = plt.subplots( 1, 3, figsize=(15,3), sharey=True, sharex=True )
ax[0].hist( cand_gnep, bins=20 )
ax[1].hist( cand_rnep, bins=20 )
ax[2].hist( cand_inep, bins=20 )
# ax[0].set_xlim([17,23])
plt.show()

fig, ax = plt.subplots( 1, 3, figsize=(15,3), sharey=True, sharex=True )
ax[0].hist( cand_gnrep, bins=10 )
ax[1].hist( cand_rnrep, bins=10 )
ax[2].hist( cand_inrep, bins=10 )
# ax[0].set_xlim([17,23])
plt.show()

In [ ]:
fig, ax = plt.subplots( 1, 3, figsize=(15,3), sharey=True, sharex=True )
ax[0].plot( cand_gnobj, cand_gnrep, 'o', ms=2, alpha=0.05, mew=0, color='darkgreen' )
ax[1].plot( cand_rnobj, cand_rnrep, 'o', ms=2, alpha=0.05, mew=0, color='darkorange' )
ax[2].plot( cand_inobj, cand_inrep, 'o', ms=2, alpha=0.05, mew=0, color='red' )
# ax[0].set_xlim([17,23])
plt.show()

In [ ]:
gx = np.where( (cand_gnobj > 15) & (cand_gnrep == 3) )[0]
rx = np.where( (cand_rnobj > 15) & (cand_rnrep == 3) )[0]
ix = np.where( (cand_inobj > 15) & (cand_inrep == 3) )[0]

fig, ax = plt.subplots( 1, 3, figsize=(15,3), sharey=True, sharex=True )
ax[0].hist( cand_gpk, bins=50, color='darkgreen' )
ax[1].hist( cand_rpk, bins=50, color='darkorange' )
ax[2].hist( cand_ipk, bins=50, color='red' )
ax[0].set_xlim([17,23])
plt.show()

fig, ax = plt.subplots( 1, 3, figsize=(15,3), sharey=True, sharex=True )
ax[0].hist( cand_gpk[gx], bins=50, color='darkgreen' )
ax[1].hist( cand_rpk[rx], bins=50, color='darkorange' )
ax[2].hist( cand_ipk[ix], bins=50, color='red' )
ax[0].set_xlim([17,23])
plt.show()

del gx,rx,ix

In [ ]:
tx = np.where( (cand_gnobj > 15) & (cand_rnobj > 15) & (cand_inobj > 15) & \
               (cand_gnrep == 3) & (cand_rnrep == 3) & (cand_inrep == 3) )[0]
print(len(tx))

fig, ax = plt.subplots( 5, 6, figsize=(20,15) )
i = 0
j = 0
for x in tx:
    ox = np.where( obj2_candid == cand_id[x] )[0]
    gx = np.where( obj2_filter[ox] == 'g' )[0]
    rx = np.where( obj2_filter[ox] == 'r' )[0]
    ix = np.where( obj2_filter[ox] == 'i' )[0]
    ax[i,j].plot( obj2_mjd[ox[gx]], obj2_mag[ox[gx]], 'o', ms=3, mew=0, alpha=0.4, color='darkgreen' )
    ax[i,j].plot( obj2_mjd[ox[rx]], obj2_mag[ox[rx]], 'o', ms=3, mew=0, alpha=0.4, color='darkorange' )
    ax[i,j].plot( obj2_mjd[ox[ix]], obj2_mag[ox[ix]], 'o', ms=3, mew=0, alpha=0.4, color='red' )
    ax[i,j].invert_yaxis()
    ax[i,j].set_title(cand_id[x])
    j += 1
    if j == 6:
        j = 0
        i += 1
plt.tight_layout()
plt.show()

In [ ]:
targets = np.asarray( ['DC21bgiwr', 'DC21qcpw', 'DC21sbdf', 'DC22eojsc', 'DC22hlnrv', 'DC22hlnsy'], dtype='str' )

print('%-10s   %-10s  %-10s   %-10s  %-10s' % ('name','ra (deg)','dec (deg)','ra_err (")','dec_err (")'))
for x in range(6):
    ox = np.where( obj2_candid == targets[x] )[0]
    print('%-10s   %10.6f  %10.6f   %10.3f %10.3f' % 
          (targets[x], \
          np.round(np.mean(obj2_ra[ox]),6), np.round(np.mean(obj2_dec[ox]),6), \
          np.round(np.std(obj2_ra[ox])*3600.,3), np.round(np.std(obj2_dec[ox])*3600.,3)))

In [ ]:
fig, ax = plt.subplots( 2, 3, figsize=(20,7) )
i = 0
j = 0
for x in range(6):
    ox = np.where( obj2_candid == targets[x] )[0]
    gx = np.where( obj2_filter[ox] == 'g' )[0]
    rx = np.where( obj2_filter[ox] == 'r' )[0]
    ix = np.where( obj2_filter[ox] == 'i' )[0]
    ax[i,j].plot( obj2_mjd[ox[gx]], obj2_mag[ox[gx]], 'o', ms=3, mew=0, alpha=0.4, color='darkgreen' )
    ax[i,j].plot( obj2_mjd[ox[rx]], obj2_mag[ox[rx]], 'o', ms=3, mew=0, alpha=0.4, color='darkorange' )
    ax[i,j].plot( obj2_mjd[ox[ix]], obj2_mag[ox[ix]], 'o', ms=3, mew=0, alpha=0.4, color='red' )
    ax[i,j].invert_yaxis()
    ax[i,j].set_title(targets[x])
    j += 1
    if j == 3:
        j = 0
        i += 1
plt.tight_layout()
plt.show()

In [ ]:
temp1 = Time(59725, format='mjd')
print(temp1.isot)